<a href="https://colab.research.google.com/github/Jaimemorillo/ShouldIwatchThisMovie/blob/master/methods.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from numpy.random import seed
seed(9)

import tensorflow as tf
tf.random.set_seed(9)

import nltk
from nltk.tokenize import WordPunctTokenizer
from nltk import word_tokenize
from nltk.stem import SnowballStemmer
import keras
import re
import string
from collections import Counter

nltk.download("popular")
nltk.download('stopwords')

import matplotlib.pyplot as plt
plt.style.use('seaborn-white')

import seaborn as sns
sns.set_style("white")
sns.set_style("ticks")
sns.set_context("notebook")

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, precision_score, recall_score, roc_curve, roc_auc_score

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras import layers
from tensorflow.keras import regularizers

In [ ]:
def read_data(file, overview_column='overview', label_column='like'):
  df = pd.read_csv(file, sep='#',encoding='utf-8', lineterminator='\n')
  df = df.rename(columns={overview_column: 'overview', label_column: 'like'})
  df['text_array'] = df.overview.str.split(" ")
  df['n_words'] = df['text_array'].apply(lambda x: len(x))
  df = df.drop(columns=['text_array'])
  df = df[df['n_words']>=5]

  return df

In [ ]:
def preprocessing(sentece):
  # Normalización
  def normalize(x):
    x = x.lower()
    replacements = (
        ("á", "a"),
        ("é", "e"),
        ("í", "i"),
        ("ó", "o"),
        ("ú", "u"),
        ("ñ", "n")
    )
    for a, b in replacements:
        x = x.replace(a, b)

    x = x.translate(str.maketrans('','',string.punctuation))
    x = x.translate(str.maketrans('','','ªº¡¿'))   
    x = x.replace("  ", " ") 
    return x

  # Stop_words
  stop_words = pd.read_csv("gdrive/My Drive/TFG/stopwords-es.txt",header=None)
  stop_words = stop_words[0].tolist()
  stop_words = [normalize(word) for word in stop_words]

  def delete_stop_words(x):
    words = x.split(' ')
    words = [word for word in words if word not in stop_words]
    x = str(' '.join(words))
    return x

  # Steaming
  stemmer = SnowballStemmer("spanish", ignore_stopwords=True)

  def stem_sentence(sentence):
    stemmed_text = [stemmer.stem(word) for word in word_tokenize(sentence)]
    return " ".join(stemmed_text)

  sentece = normalize(sentece)
  sentece = delete_stop_words(sentece)
  sentece = stem_sentence(sentece)

  return sentece

In [ ]:
def split_train_test(df):
  X = df['overview']
  y = df['like']

  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, stratify=y, random_state=10)

  return X_train, X_test, y_train, y_test

In [1]:
def tokenize(X_train, X_test, maxlen=None, test_mode=False):
  tokenizer = Tokenizer()
  tokenizer.fit_on_texts(X_train)

  X_train = tokenizer.texts_to_sequences(X_train) 
  X_test = tokenizer.texts_to_sequences(X_test)

  real_maxlen = len(max(X_train, key=len))
  if maxlen is None:
    maxlen = real_maxlen

  vocab_size = len(tokenizer.word_index) + 1 # Adding 1 because of reserved 0 index for future padding

  if test_mode is False:
    # seaborn histogram
    n_words_train = np.array([len(i) for i in X_train])
    sns.distplot(n_words_train, hist=True, kde=False, 
                bins=20,
                hist_kws={'edgecolor':'tab:blue', 'linewidth': 2})
    # Add labels
    plt.title('Histograma número de palabras')
    plt.xlabel('Número palabras')
    plt.ylabel('Películas')
    mean = n_words_train.mean()
    plt.vlines(mean, 0, 215, color='crimson', ls=':')

  X_train = pad_sequences(X_train, padding='pre', maxlen=maxlen, truncating='post')
  X_test = pad_sequences(X_test, padding='pre', maxlen=maxlen, truncating='post')

  return X_train, X_test, vocab_size, real_maxlen

In [ ]:
def my_model(vocab_size, maxlen=90, embedding_dim=64, lr=0.001, epochs=40):
  inputs = layers.Input(shape=(maxlen,))

  # Embedding
  x = layers.Embedding(input_dim = vocab_size, output_dim = embedding_dim)(inputs)

  # CNN
  x = layers.Conv1D(32, 3, padding="same", activation="relu", strides=1)(x)
  x = layers.BatchNormalization()(x)
  x = layers.MaxPooling1D(pool_size=2, strides=2)(x)

  x = layers.Conv1D(32, 3, padding="same", activation="relu", strides=1)(x)
  x = layers.BatchNormalization()(x)
  x = layers.MaxPooling1D(pool_size=2, strides=2)(x)

  # LSTM
  x = layers.Bidirectional(layers.LSTM(16))(x)
  x = layers.BatchNormalization()(x)

  # FNN
  x = layers.Dense(16, activation="relu")(x)
  x = layers.BatchNormalization()(x)
  x = layers.Dropout(0.5)(x)

  x = layers.Dense(8, activation="relu")(x)
  x = layers.BatchNormalization()(x)
  x = layers.Dropout(0.5)(x)

  # Output layer, 1 neuron with sigmoid:
  predictions = layers.Dense(1, activation="sigmoid", name="predictions")(x)

  model = tf.keras.Model(inputs, predictions)
  model.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=lr, decay = lr/epochs), metrics=['accuracy'])

  return model

In [1]:
def train_model(model, X_train, y_train, X_test, y_test, epochs, batch_size):
  callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
  history = model.fit(X_train, y_train,
                    epochs=epochs,
                    verbose=True,
                    validation_data=(X_test, y_test),
                    batch_size=batch_size,
                    callbacks=[callback])
  
  return history, model

In [ ]:
def plot_history(history):
  fit_history = pd.DataFrame(history.history)
  fit_history['epoch'] = fit_history.index + 1
  fit_history = fit_history.round(2)

  fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(13, 4.25))

  # Loss
  fit_history.plot(x='epoch', y=['loss', 'val_loss'], kind='line', ylabel='loss', title='Loss', marker='.', ax=ax1)

  # Acc
  fit_history.plot(x='epoch', y=['accuracy', 'val_accuracy'], kind='line', ylabel='acc', title='Accuracy', marker='.', ax=ax2)

  x = fit_history[fit_history['val_accuracy']==fit_history['val_accuracy'].max()]

  first_max_acc = fit_history[fit_history['val_accuracy']==fit_history['val_accuracy'].max()].iloc[0]
  x = first_max_acc['epoch']
  y = first_max_acc['val_accuracy']
  z = first_max_acc['accuracy']

  ax2.text(x = x - 0.35, # x-coordinate position of data label
          y = y - 0.06, # y-coordinate position of data label
          s = '{:.2f}'.format(y), # data label
          color = 'tab:orange') # set colour of line
  ax2.text(x = x - 0.35, # x-coordinate position of data label
          y = z - 0.05, # y-coordinate position of data label
          s = '{:.2f}'.format(z), # data label
          color = 'tab:blue') # set colour of line

In [ ]:
def plot_metrics(model, X_test, y_test):
  y_pred = (model.predict(X_test) > 0.5).astype("int32")
  y_true = y_test
  print('Acc: ' + str(accuracy_score(y_true, y_pred).round(2)))
  print('Precision: ' + str(precision_score(y_true, y_pred).round(2)))
  print('Recall: ' + str(recall_score(y_true, y_pred).round(2)))
  print('F1: ' + str(f1_score(y_true, y_pred).round(2)))
  print('Conf. Matrix: ')
  print(confusion_matrix(y_true, y_pred))

In [ ]:
def get_roc_curve(model, X_test, y_test):
  y_pred_keras = model.predict(X_test).flatten()
  y_pred = (model.predict(X_test) > 0.5).astype("int32")
  y_true = y_test
  fpr, tpr, thresholds = roc_curve(y_true, y_pred_keras)
  roc_auc = roc_auc_score(y_true, y_pred)

  def plot_roc_curve(fpr,tpr): 
    plt.title('ROC Curve')
    plt.plot(fpr, tpr, color='tab:blue', label = 'AUC = %0.2f' % roc_auc)
    plt.legend(loc = 'lower right')
    plt.plot([0, 1], [0, 1],'--', color='tab:orange')
    plt.xlim([0, 1])
    plt.ylim([0, 1])
    plt.ylabel('True Positive Rate')
    plt.xlabel('False Positive Rate')
    plt.show()   
    
  plot_roc_curve(fpr,tpr) 
  print('AUC: ' + str(roc_auc.round(4)))

In [ ]:
def plot_confusion_matrix(model, X_test, y_test):
  y_pred = (model.predict(X_test) > 0.5).astype("int32")
  y_true = y_test
  matrix_con = confusion_matrix(y_true, y_pred)
  sns.heatmap(matrix_con, annot=True, fmt='g')